# Run Configuration Management

This notebook demonstrates safe run configuration management with validation safeguards.

## What You'll Learn

- Why run configuration management matters
- The critical HMS auto-deletion problem
- Querying run configurations and components
- Modifying run metadata (description, log file, DSS output)
- Modifying run components with validation (basin, met, control)
- Direct file modification vs. project initialization

## The Critical Problem

**HEC-HMS automatically deletes runs with invalid component references when opening a project.**

If you modify a `.run` file to reference a basin, met model, or control spec that doesn't exist:
1. HMS will silently remove the run from the project
2. No warning or error message is provided
3. The run permanently disappears

**Solution**: The `HmsRun.set_*()` methods validate component existence BEFORE modifying files.

**Estimated Time**: 10-15 minutes

In [1]:
# pip install hms-commander

**For Development**: If working on hms-commander source code, use the `hmscmdr_local` conda environment (editable install) instead of pip install.

In [2]:
from pathlib import Path
from hms_commander import (
    HmsExamples,
    init_hms_project,
    HmsRun,
    HmsPrj
)

print("hms-commander loaded")

hms-commander loaded


## 1. Initialize Example Project

In [3]:
# Extract the tifton example project
project_path = HmsExamples.extract_project(
    "tifton",
    output_path=Path.cwd() / 'hms_example_projects' / 'tifton_run_management'
)

# Initialize project
hms = init_hms_project(project_path)

print(f"Initialized: {hms.project_name}")
print(f"Version: {hms.hms_version}")

2026-01-08 13:19:54 - hms_commander.HmsExamples - INFO - Found HMS 4.10 at C:\Program Files\HEC\HEC-HMS\4.10


2026-01-08 13:19:54 - hms_commander.HmsExamples - INFO - Found HMS 4.11 at C:\Program Files\HEC\HEC-HMS\4.11


2026-01-08 13:19:54 - hms_commander.HmsExamples - INFO - Found HMS 4.12 at C:\Program Files\HEC\HEC-HMS\4.12


2026-01-08 13:19:54 - hms_commander.HmsExamples - INFO - Found HMS 4.13 at C:\Program Files\HEC\HEC-HMS\4.13


2026-01-08 13:19:54 - hms_commander.HmsExamples - INFO - Found HMS 4.4.1 at C:\Program Files\HEC\HEC-HMS\4.4.1


...

2026-01-08 13:19:55 - hms_commander.HmsPrj - INFO -   Control specs: 1


2026-01-08 13:19:55 - hms_commander.HmsPrj - INFO -   Simulation runs: 1


2026-01-08 13:19:55 - hms_commander.HmsPrj - INFO -   Gages: 2


2026-01-08 13:19:55 - hms_commander.HmsPrj - INFO -   Paired data tables: 0


Initialized: tifton
Version: 4.13


## 2. View Available Components

Before modifying runs, check what components exist in the project.

In [4]:
print("Available Components:")
print("=" * 60)
print(f"Basins:        {hms.list_basin_names()}")
print(f"Met Models:    {hms.list_met_names()}")
print(f"Control Specs: {hms.list_control_names()}")
print(f"Runs:          {hms.list_run_names()}")
print(f"Gages:         {hms.list_gage_names()}")

Available Components:
Basins:        ['Tifton']
Met Models:    ['Tifton Hyetograph']
Control Specs: ['Jan1-Jun30 1970']
Runs:          ['1970 simulation']
Gages:         ['Gage 74006', 'Gage 38']


## 3. View Current Run Configurations

In [5]:
# Display all runs using run_df
print("Current Run Configurations:")
print("=" * 60)
hms.run_df[['name', 'basin_model', 'met_model', 'control_spec', 'dss_file']]

Current Run Configurations:


,name,basin_model,met_model,control_spec,dss_file
0,1970 simulation,Tifton,Tifton Hyetograph,Jan1-Jun30 1970,1970_simulation.dss


### Understanding get_dss_config() Return Keys

The `HmsRun.get_dss_config()` method returns a dictionary with these keys:

| Key | Description | Example |
|-----|-------------|--------|
| `basin_model` | Basin model name | `'Tifton'` |
| `met_model` | Meteorologic model name | `'Tifton'` |
| `control_spec` | Control specification name | `'Jan70-Jun70'` |
| `dss_file` | DSS output file path | `'tifton.dss'` |
| `log_file` | Log file path | `'tifton.log'` |
| `description` | Run description (if set) | `'Design storm analysis'` |
| `start_tab` | Starting tab in HMS GUI | `'1'` |

In [6]:
# Get detailed configuration for first run
run_name = hms.list_run_names()[0]
config = HmsRun.get_dss_config(run_name, hms_object=hms)

print(f"Configuration for '{run_name}':")
print("=" * 60)
for key, value in config.items():
    print(f"  {key:20s}: {value}")

2026-01-08 13:19:55 - hms_commander.HmsRun - INFO - Retrieved DSS config for run '1970 simulation': 1970_simulation.dss


Configuration for '1970 simulation':
  dss_file            : 1970_simulation.dss
  dss_path            : C:\GH\hms-commander\examples\hms_example_projects\tifton_run_management\tifton\1970_simulation.dss
  log_file            : 1970_simulation.log
  time_series_output  : Save All
  basin_model         : Tifton
  met_model           : Tifton Hyetograph
  control_spec        : Jan1-Jun30 1970
  run_file            : C:\GH\hms-commander\examples\hms_example_projects\tifton_run_management\tifton\tifton.run
  description         : Jan to Jun 1970


## 4. Modifying Run Metadata

Update run description, log file, and DSS output file.

### Set Run Description

In [7]:
# Update run description
HmsRun.set_description(
    run_name=run_name,
    description="Notebook 04: Run management demonstration",
    hms_object=hms
)
print(f"[OK] Updated description for '{run_name}'")

2026-01-08 13:19:55 - hms_commander.HmsRun - INFO - Retrieved DSS config for run '1970 simulation': 1970_simulation.dss


2026-01-08 13:19:55 - hms_commander.HmsRun - INFO - Updated description for run '1970 simulation' in C:\GH\hms-commander\examples\hms_example_projects\tifton_run_management\tifton\tifton.run


[OK] Updated description for '1970 simulation'


### Set Log File Path

In [8]:
# Update log file path
HmsRun.set_log_file(
    run_name=run_name,
    log_file="run_management_demo.log",
    hms_object=hms
)
print(f"[OK] Updated log file for '{run_name}'")

2026-01-08 13:19:55 - hms_commander.HmsRun - INFO - Retrieved DSS config for run '1970 simulation': 1970_simulation.dss


2026-01-08 13:19:55 - hms_commander.HmsRun - INFO - Updated log file for run '1970 simulation' to 'run_management_demo.log' in C:\GH\hms-commander\examples\hms_example_projects\tifton_run_management\tifton\tifton.run


[OK] Updated log file for '1970 simulation'


### Set DSS Output File

In [9]:
# Update DSS output file
HmsRun.set_dss_file(
    run_name=run_name,
    dss_file="run_management_output.dss",
    hms_object=hms,
    update_log_file=True  # Also updates log file to match
)
print(f"[OK] Updated DSS output file for '{run_name}'")

2026-01-08 13:19:55 - hms_commander.HmsRun - INFO - Retrieved DSS config for run '1970 simulation': 1970_simulation.dss


2026-01-08 13:19:55 - hms_commander.HmsRun - INFO - Updated DSS output for run '1970 simulation' to 'run_management_output.dss'


[OK] Updated DSS output file for '1970 simulation'


### Why Reinitialize?

After modifying HMS files, you should **reinitialize the project** to refresh the DataFrames:

**Why this is needed:**
1. **DataFrames are snapshots** - They are created once during `init_hms_project()`
2. **File modifications don't update DataFrames** - Changes write to disk but not to memory
3. **Stale data causes confusion** - DataFrames may show old values

**When to reinitialize:**
- After any `set_*()` call if you need to verify changes via DataFrames
- Before running simulations after configuration changes
- When you need to query the updated state

**Performance note:** Reinitializing is fast (typically <1 second for most projects).

In [10]:
# Reinitialize to refresh DataFrames
hms = init_hms_project(project_path)

# Verify changes
updated_config = HmsRun.get_dss_config(run_name, hms_object=hms)

print(f"Updated Configuration for '{run_name}':")
print("=" * 60)
print(f"  Description: {updated_config.get('description', 'N/A')}")
print(f"  Log File:    {updated_config['log_file']}")
print(f"  DSS File:    {updated_config['dss_file']}")

2026-01-08 13:19:55 - hms_commander.HmsPrj - INFO - HMS project initialized: tifton


2026-01-08 13:19:55 - hms_commander.HmsPrj - INFO -   Version: 4.13


2026-01-08 13:19:55 - hms_commander.HmsPrj - INFO -   Basin models: 1


2026-01-08 13:19:55 - hms_commander.HmsPrj - INFO -   Met models: 1


2026-01-08 13:19:55 - hms_commander.HmsPrj - INFO -   Control specs: 1


2026-01-08 13:19:55 - hms_commander.HmsPrj - INFO -   Simulation runs: 1


2026-01-08 13:19:55 - hms_commander.HmsPrj - INFO -   Gages: 2


2026-01-08 13:19:55 - hms_commander.HmsPrj - INFO -   Paired data tables: 0


2026-01-08 13:19:55 - hms_commander.HmsRun - INFO - Retrieved DSS config for run '1970 simulation': run_management_output.dss


Updated Configuration for '1970 simulation':
  Description: Notebook 04: Run management demonstration
  Log File:    run_management_output.log
  DSS File:    run_management_output.dss


## 5. Modifying Run Components (with Validation)

**CRITICAL**: When changing which basin, met model, or control spec a run uses, validation prevents HMS from deleting the run.

### Set Basin Model (with Validation)

In [11]:
# Get available basins
basin_names = hms.list_basin_names()
print(f"Available basins: {basin_names}")

if basin_names:
    # Set basin model (validates it exists first!)
    try:
        HmsRun.set_basin(
            run_name=run_name,
            basin_model=basin_names[0],
            hms_object=hms
        )
        print(f"[OK] Set basin to '{basin_names[0]}'")
        print("     Validation passed - basin exists in project")
    except ValueError as e:
        print(f"[ERROR] {e}")

2026-01-08 13:19:55 - hms_commander.HmsRun - INFO - Retrieved DSS config for run '1970 simulation': run_management_output.dss


2026-01-08 13:19:55 - hms_commander.HmsRun - INFO - Basin already set to 'Tifton' for run '1970 simulation'


Available basins: ['Tifton']
[OK] Set basin to 'Tifton'
     Validation passed - basin exists in project


### Set Meteorologic Model (with Validation)

In [12]:
# Get available met models
met_names = hms.list_met_names()
print(f"Available met models: {met_names}")

if met_names:
    # Set met model (validates it exists first!)
    try:
        HmsRun.set_precip(
            run_name=run_name,
            met_model=met_names[0],
            hms_object=hms
        )
        print(f"[OK] Set met model to '{met_names[0]}'")
        print("     Validation passed - met model exists in project")
    except ValueError as e:
        print(f"[ERROR] {e}")

2026-01-08 13:19:55 - hms_commander.HmsRun - INFO - Retrieved DSS config for run '1970 simulation': run_management_output.dss


2026-01-08 13:19:55 - hms_commander.HmsRun - INFO - Met model already set to 'Tifton Hyetograph' for run '1970 simulation'


Available met models: ['Tifton Hyetograph']
[OK] Set met model to 'Tifton Hyetograph'
     Validation passed - met model exists in project


### Set Control Specification (with Validation)

In [13]:
# Get available control specs
control_names = hms.list_control_names()
print(f"Available control specs: {control_names}")

if control_names:
    # Set control spec (validates it exists first!)
    try:
        HmsRun.set_control(
            run_name=run_name,
            control_spec=control_names[0],
            hms_object=hms
        )
        print(f"[OK] Set control spec to '{control_names[0]}'")
        print("     Validation passed - control spec exists in project")
    except ValueError as e:
        print(f"[ERROR] {e}")

2026-01-08 13:19:55 - hms_commander.HmsRun - INFO - Retrieved DSS config for run '1970 simulation': run_management_output.dss


2026-01-08 13:19:55 - hms_commander.HmsRun - INFO - Control spec already set to 'Jan1-Jun30 1970' for run '1970 simulation'


Available control specs: ['Jan1-Jun30 1970']
[OK] Set control spec to 'Jan1-Jun30 1970'
     Validation passed - control spec exists in project


## 6. Demonstrating Validation Protection

**This is the most important feature!** Validation prevents HMS from silently deleting runs.

### Example: Invalid Basin (Caught by Validation)

In [14]:
# Try to set a non-existent basin
try:
    HmsRun.set_basin(
        run_name=run_name,
        basin_model="NonExistentBasin",
        hms_object=hms
    )
    print("[ERROR] Should have failed validation!")
except ValueError as e:
    print("[OK] Validation prevented invalid configuration:")
    print("=" * 60)
    print(str(e))
    print("\n" + "=" * 60)
    print("THIS IS CRITICAL!")
    print("Without validation, HMS would silently delete this run")
    print("when you next open the project. No warning, no error.")
    print("The run would simply disappear.")
    print("=" * 60)

[OK] Validation prevented invalid configuration:
Basin 'NonExistentBasin' not found in project. Available basins: ['Tifton']. HMS will delete runs with invalid basin references on project open!

THIS IS CRITICAL!
Without validation, HMS would silently delete this run
when you next open the project. No warning, no error.
The run would simply disappear.


### Example: Invalid Met Model (Caught by Validation)

In [15]:
# Try to set a non-existent met model
try:
    HmsRun.set_precip(
        run_name=run_name,
        met_model="NonExistentMet",
        hms_object=hms
    )
    print("[ERROR] Should have failed validation!")
except ValueError as e:
    print("[OK] Validation prevented invalid configuration:")
    print("=" * 60)
    print(str(e))

[OK] Validation prevented invalid configuration:
Met model 'NonExistentMet' not found in project. Available met models: ['Tifton Hyetograph']. HMS will delete runs with invalid met references on project open!


## 7. Verify DSS Output Files Exist

In [16]:
# Check which runs have generated DSS output files
dss_status = HmsRun.verify_dss_outputs(hms_object=hms)

print("DSS Output File Status:")
print("=" * 60)
for run, info in dss_status.items():
    status = "[EXISTS]" if info['exists'] else "[NOT FOUND]"
    print(f"{status} {run}: {info['dss_file']}")

2026-01-08 13:19:55 - hms_commander.HmsRun - INFO - Retrieved DSS config for run '1970 simulation': run_management_output.dss


2026-01-08 13:19:55 - hms_commander.HmsRun - INFO - Listed outputs for 1 runs


2026-01-08 13:19:55 - hms_commander.HmsRun - INFO - DSS output verification: 0/1 files exist


DSS Output File Status:
[NOT FOUND] 1970 simulation: run_management_output.dss


## 8. Direct File Modification (Advanced)

For advanced users: modify `.run` files directly without project initialization.

**Trade-offs**:
- Faster - No project initialization overhead
- Flexible - Works on any .run file
- No validation - You must ensure components exist
- Risk - Invalid configurations will cause HMS to delete runs

In [17]:
# Get path to run file
run_file = hms.project_folder / (hms.project_folder.name + ".run")
print(f"Run file: {run_file}")

# Modify directly without project initialization
HmsRun.set_description_direct(
    run_file_path=run_file,
    run_name=run_name,
    description="Modified via direct file access (no project init)"
)
print("[OK] Modified run file directly")
print("\nNOTE: No validation was performed!")
print("Use direct methods only if you're certain components exist.")

2026-01-08 13:19:55 - hms_commander.HmsRun - INFO - Updated description for run '1970 simulation' in C:\GH\hms-commander\examples\hms_example_projects\tifton_run_management\tifton\tifton.run


Run file: C:\GH\hms-commander\examples\hms_example_projects\tifton_run_management\tifton\tifton.run
[OK] Modified run file directly

NOTE: No validation was performed!
Use direct methods only if you're certain components exist.


## 9. Best Practices Summary

### DO

1. **Always use accessor methods** to check component availability:
   ```python
   basins = hms.list_basin_names()
   if "MyBasin" in basins:
       HmsRun.set_basin(run_name, "MyBasin", hms_object=hms)
   ```

2. **Use validated methods** for component assignment:
   ```python
   HmsRun.set_basin(...)   # Validates!
   HmsRun.set_precip(...)  # Validates!
   HmsRun.set_control(...) # Validates!
   ```

3. **Reinitialize after modifications** to refresh DataFrames:
   ```python
   HmsRun.set_description(...)
   hms = init_hms_project(project_path)  # Refresh!
   ```

4. **Use separate DSS files** for scenario comparison:
   ```python
   HmsRun.set_dss_file("Baseline", "baseline.dss", hms_object=hms)
   HmsRun.set_dss_file("Updated", "updated.dss", hms_object=hms)
   ```

### DON'T

1. Don't use direct methods for component assignment unless you're certain components exist
2. Don't skip validation - HMS will delete runs with invalid components
3. Don't forget to reinitialize after modifications
4. Don't assume error messages - HMS deletes runs silently

## Summary

You've learned:

| Feature | Method | Purpose |
|---------|--------|--------|
| Query config | `HmsRun.get_dss_config()` | View current run settings |
| Set description | `HmsRun.set_description()` | Update run metadata |
| Set DSS output | `HmsRun.set_dss_file()` | Configure output file |
| Set basin | `HmsRun.set_basin()` | Change basin model (validated) |
| Set met | `HmsRun.set_precip()` | Change met model (validated) |
| Set control | `HmsRun.set_control()` | Change control spec (validated) |
| Verify outputs | `HmsRun.verify_dss_outputs()` | Check DSS file existence |

## Next Steps

- **05_clone_workflow.ipynb**: Clone and compare workflow for QAQC
- **06_results_dss.ipynb**: Extract results from DSS files
- **07_execution_jython.ipynb**: Advanced execution patterns